In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import classification_report, confusion_matrix

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Set up important landmarks and functions

In [2]:
# Determine important landmarks for lunge
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

TRAIN_SET_PATH  = "./err.train.csv"
TEST_SET_PATH  = "./err.test.csv"

In [3]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_dataset.csv", sep=',', encoding='utf-8', index=False)

## 2. Describe and process data

In [4]:
# load dataset
df = describe_dataset(TRAIN_SET_PATH)

# Categorizing label
df.loc[df["label"] == "L", "label"] = 0
df.loc[df["label"] == "C", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_v', 'right_foot_index_x', 'right_foot_index_y', 'right_foot_index_z', 'right_foot_index_v']
Number of rows: 9736 
Number of columns: 53

Labels: 
C    4931
L    4805
Name: label, dtype: int64

Missing values: 

In [5]:
# Standard Scaling of features
with open("./model/err_input_scaler.pkl", "rb") as f2:
    input_scaler = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(input_scaler.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

## 3. Train model

In [10]:
model = Sequential()
model.add(Dense(52, input_dim = 52, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(52, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(14, activation = "relu"))
model.add(Dense(2, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 52)                2756      
                                                                 
 dropout_2 (Dropout)         (None, 52)                0         
                                                                 
 dense_5 (Dense)             (None, 52)                2756      
                                                                 
 dropout_3 (Dropout)         (None, 52)                0         
                                                                 
 dense_6 (Dense)             (None, 14)                742       
                                                                 
 dense_7 (Dense)             (None, 2)                 30        
                                                                 
Total params: 6,284
Trainable params: 6,284
Non-traina

In [7]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

In [11]:
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Epoch 1/100


2022-11-21 16:32:53.614290: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/779 [..............................] - ETA: 4:56 - loss: 0.8906 - accuracy: 0.5000

2022-11-21 16:32:53.850537: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


779/779 [==============================] - ETA: 0s - loss: 0.0416 - accuracy: 0.9873

2022-11-21 16:33:00.892941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


779/779 [==============================] - 8s 10ms/step - loss: 0.0416 - accuracy: 0.9873 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 2/100
779/779 [==============================] - 8s 10ms/step - loss: 0.0099 - accuracy: 0.9978 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 3/100
779/779 [==============================] - 8s 10ms/step - loss: 0.0296 - accuracy: 0.9968 - val_loss: 3.6784e-04 - val_accuracy: 1.0000
Epoch 4/100
779/779 [==============================] - 8s 10ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 2.6883e-04 - val_accuracy: 1.0000
Epoch 5/100
779/779 [==============================] - 7s 9ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 8.6227e-05 - val_accuracy: 1.0000
Epoch 6/100
779/779 [==============================] - 7s 9ms/step - loss: 0.0456 - accuracy: 0.9959 - val_loss: 0.0123 - val_accuracy: 0.9902
Epoch 7/100
779/779 [==============================] - 7s 9ms/step - loss: 0.0280 - accuracy: 0.9974 - val_loss: 3.3980e-06 - val_accuracy

## 4. Model Evaluation

### 4.1. Train set

In [12]:
predict_x = model.predict(x_test) 
y_pred_class = np.argmax(predict_x, axis=1)

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)

confusion_matrix(y_test_class, y_pred_class)

 1/61 [..............................] - ETA: 0s

2022-11-21 16:36:18.027072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


61/61 [==============================] - 0s 2ms/step


array([[971,   0],
       [  0, 977]])

In [13]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       971
           1       1.00      1.00      1.00       977

    accuracy                           1.00      1948
   macro avg       1.00      1.00      1.00      1948
weighted avg       1.00      1.00      1.00      1948



### 4.2. Test set evaluation

In [14]:
test_df = pd.read_csv(TEST_SET_PATH)

# Categorizing label
test_df.loc[test_df["label"] == "L", "label"] = 0
test_df.loc[test_df["label"] == "C", "label"] = 1

In [15]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(input_scaler.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [16]:
predict_x = model.predict(test_x) 
y_pred_class = np.argmax(predict_x, axis=1)
y_test_class = np.argmax(test_y_cat, axis=1)

confusion_matrix(y_test_class, y_pred_class)

20/20 [==============================] - 0s 8ms/step


2022-11-21 16:36:41.831014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[344,   0],
       [255,  23]])

In [17]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.57      1.00      0.73       344
           1       1.00      0.08      0.15       278

    accuracy                           0.59       622
   macro avg       0.79      0.54      0.44       622
weighted avg       0.76      0.59      0.47       622



## 5. Dump Model

In [18]:
# Dump the best model to a pickle file
with open("./model/err_lunge_dp.pkl", "wb") as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://4b141eb3-ab3a-4c7e-bb97-2377a5bf6ae5/assets
